In [1]:
using DataStructures # Just for DefaultDict

function read_reports(filename::String)::Dict{Int64,Vector{Vector{Int64}}}
    scanner_dict = Dict{Int64,Vector{Vector{Int64}}}([])
    scanner_id = nothing
    for line in readlines(filename)
        if startswith(line, "---")
            scanner_id = parse(Int, split(line, ' ')[3])
            scanner_dict[scanner_id] = []
        elseif length(line) > 0
            push!(scanner_dict[scanner_id], parse.(Int, collect(split(line, ','))))
        end
    end
    return scanner_dict
end

function pair_diffs(scan::Vector{Vector{Int64}})::Dict{Tuple{Int64,Int64},Int64}
    # Translation/reflection/rotation invariant for comparison
    hash = Dict{Tuple{Int64,Int64},Int64}([])
    for (ib1, b1) in enumerate(scan)
        for (ib2, b2) in enumerate(scan)
            if b1 ≠ b2
                hash[ib1, ib2] = sum((b1 - b2).^2)
            end
        end
    end
    return hash
end

function scan_overlap(scan1::Vector{Vector{Int64}}, 
                      scan2::Vector{Vector{Int64}}, 
                      pd1::Dict{Tuple{Int64,Int64},Int64}, 
                      pd2::Dict{Tuple{Int64,Int64},Int64}, 
                      shared::Set{Int64}
                    )::Tuple{Dict{Int64,Int64},Union{Nothing,Vector{Int64}}}
    matched_pairs = DefaultDict{Int64,Vector{Tuple{Int64,Int64}}}([])
    for s in shared
        for p1 in  [k for (k,v) in pd1 if v==s]
            p2 = [k for (k,v) in pd2 if v==s]
            append!(matched_pairs[p1[1]], p2)
            append!(matched_pairs[p1[2]], p2)
        end
    end
    final_pairs = Dict{Int64,Int64}([])
    offsets = Set{Vector{Int64}}([])
    for (k, v) in matched_pairs
        flat_v = vcat([[tup...] for tup in v]...)
        uniques = unique(flat_v)
        counts = [count(x->x==uniq, flat_v) for uniq in uniques]
        most = argmax(counts)
        if counts[most] > 12
            final_pairs[k] = uniques[most]
            off = scan1[k] .- scan2[final_pairs[k]]
            push!(offsets, off)
            if length(offsets) > 1
                # Not all beacons have the same offset
                return final_pairs, nothing
            end
        end
    end
    return final_pairs, first(offsets)
end

function rot3D(n::Int64, axis::Int64)::Matrix{Int64}
    cth = Int(real(im^n))
    sth = Int(imag(im^n))
    # There is some way to do this by rolling axes
    if axis == 1
        return [1    0    0;
                0  cth -sth;
                0  sth  cth]
    elseif axis == 2
        return [cth  0 -sth;
                  0  1    0
                sth  0  cth]
    elseif axis == 3
        return [cth -sth  0;
                sth  cth  0;
                  0    0  1]
    end
end

function solve(filename)

    facings = vcat([rot3D(i, 3) for i = 0:3], [rot3D(i, 2) for i = 1:2:3]) # 6 different ways to face
    diff_ups = [rot3D(i, 1) for i=0:3] # 4 different ways to consider up
    transforms = [f * d for (f,d) in Iterators.product(facings, diff_ups)][:]

    scans = read_reports(filename)

    # Only calculate the pair diffs and set intersections once
    pd_cache = Dict{Int64,Dict{Tuple{Int64,Int64},Int64}}([])
    for (scan_id, scan) in scans
        pd_cache[scan_id] = pair_diffs(scan)
    end
    setint_cache = Dict{Tuple{Int64,Int64},Set{Int64}}([])
    for (scan_id1, scan1) in scans
        for (scan_id2, scan2) in scans
            if scan_id1 < scan_id2
                s1 = Set{Int64}(values(pd_cache[scan_id1]))
                s2 = Set{Int64}(values(pd_cache[scan_id2]))
                shared = intersect(s1, s2)
                if length(shared) >= 66 # =12*11/2, ie. atleast 12 matches
                    setint_cache[scan_id1, scan_id2] = shared
                end
            end
        end
    end
    beacons = Set{Vector{Int64}}([scans[0]...])
    searching = Dict{Int64,Tuple{Vector{Vector{Int64}}, Vector{Int64}}}([0=>(pop!(scans, 0), [0,0,0])])
    scanner_locs = Vector{Int64}[]

    while length(scans) > 0
        newly_found = Dict{Int64, Tuple{Vector{Vector{Int64}},Vector{Int64}}}([])
        for (scan_id1, (scan1, offset1)) in searching
            for (scan_id2, scan2) in scans
                tup = Tuple(sort([scan_id1, scan_id2]))
                # Not in cache, means not enough overlap
                if !haskey(setint_cache, tup)
                    continue
                end
                shared = Set{Int64}(setint_cache[tup])
                for t in transforms
                    tscan2 = Vector{Int64}[t * s for s in scan2]
                    matches, offset2 = scan_overlap(scan1, tscan2, pd_cache[scan_id1], pd_cache[scan_id2], shared)
                    if (offset2 != nothing)
                        pop!(scans, scan_id2)
                        noffset = offset2 .+ offset1
                        push!(scanner_locs, noffset)
                        newly_found[scan_id2] = (tscan2, noffset)
                        nbeacons = Set([([ts .+ noffset for ts in tscan2])...])
                        union!(beacons, nbeacons)
                        break
                    end
                end
            end
        end
        searching = newly_found
    end
    println("Number of beacons (part 1): $(length(beacons))")
    max_diff = 0
    for s1 in scanner_locs
        for s2 in scanner_locs
            if s1 ≠ s2
                dist = sum(abs.(s1 - s2))
                if dist > max_diff
                    max_diff = dist
                end
            end
        end
    end
    println("Ocean size (part 2): $max_diff")
end

solve (generic function with 1 method)

# Test

In [2]:
solve("test.txt")

Number of beacons (part 1): 79
Ocean size (part 2): 3621


# Full Input

In [3]:
@time solve("input.txt")

Number of beacons (part 1): 396
Ocean size (part 2): 11828
  0.415592 seconds (419.94 k allocations: 60.440 MiB, 3.50% gc time)
